In [7]:
# Import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn

# Only works in my notebook when called twice
%matplotlib notebook
%matplotlib notebook

# Bokeh graphs
from bokeh.io import curdoc, output_notebook, show
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, RangeTool, BoxAnnotation, HoverTool
from bokeh.models.widgets import Slider, TextInput
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [8]:
class TechnicalIndicators:
    
    def simple_moving_avg(self, df, column, window_size=15):
        return df[column].rolling(window=window_size, center=True).mean()
    
    def exponential_moving_avg(self, df, column, window_size=15):
        ema_df = df[column].shift(int(window_size/2)).ewm(span=window_size).mean()
        return self._remove_trailing_data(ema_df, window_size)
    
    def macd_line(self, df, column, ema1_window_size=12, ema2_window_size=26):
        macd_line_df = self.exponential_moving_avg(df, column, ema1_window_size)\
            - self.exponential_moving_avg(df, column, ema2_window_size)
        return macd_line_df
    
    def macd_signal(self, df, column, window_size=9):
        macd_line_df = pd.DataFrame()
        macd_line_df[column] = self.macd_line(df, column)
        return self.exponential_moving_avg(macd_line_df, column, 9)
    
    def rsi(self, df, column, window_size=15):
        delta, upward, downward, upward_ema, downward_ema = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
        delta = df[column].diff()
        delta = delta[1:]
        upward[column], downward[column] = delta, delta
        upward[column][upward[column] < 0] = 0
        downward[column][downward[column] > 0] = 0
        upward_ema[column] = self.exponential_moving_avg(upward, column, window_size)
        downward_ema[column] = self.exponential_moving_avg(downward.abs(), column, window_size)
        return 100 - 100/(1+upward_ema[column]/downward_ema[column])
    
    def stochastic_oscillator_k(self, df, column, window_size=15):
        lowest_low_period = df[column].rolling(window=window_size, center=True).min()
        highest_high_period = df[column].rolling(window=window_size, center=True).max()
        return (df[column] - lowest_low_period)/(highest_high_period - lowest_low_period)*100
    
    def stochastic_oscillator_d(self, df, k_column, window_size=3):
        return self.simple_moving_avg(df, k_column, window_size)
    
    def cci(self, df, close, high, low, window_size=15):
        typical_price = pd.DataFrame()
        typical_price[close] = self._typical_price(df, close, high, low)
        mean_deviation = typical_price[close].rolling(window= window_size, center=True).apply(self._mad, raw=True)
        return (typical_price[close] - self.simple_moving_avg(df, close, window_size))\
                /(0.015*mean_deviation)
    
    def money_flow(self, df, close, high, low, volume):
        typical_price = pd.DataFrame()
        typical_price[close] = self._typical_price(df, close, high, low)
        return typical_price[close] * df[volume]

    def _mad(self, x):
         return np.fabs(x - x.mean()).mean()

    def _typical_price(self, df, close, high, low):
        return (df[close] + df[high] + df[low])/3
    
    def _remove_trailing_data(self, df, window_size):
        return df.shift(-int(window_size/2)*2).shift(int(window_size/2))

In [9]:
# Obtain stock data
from pandas_datareader.av.time_series import AVTimeSeriesReader
import json

class Stock:
    '''Only works if api key for alpha vantage is provided in config.json'''
    def __init__(self, symbol, start='1/1/1800'):
        # Load config settings
        with open('config.json') as f:
            config = json.load(f)
            
        self.alpha_vantage_key = config['alpha_vantage_key']
        self.symbol = symbol
        self.start = start
        self.df = self._get_stock_data()
        self.indicator_df = self._create_indicator_columns()
        
    def _get_stock_data(self):
        ts = AVTimeSeriesReader(symbols=self.symbol, start=self.start, function='TIME_SERIES_DAILY', api_key=self.alpha_vantage_key)
        df = pd.DataFrame(ts.read())
        ts.close()
        df.index = pd.to_datetime(df.index)
        return df
    
    def _create_indicator_columns(self):
        close = 'close'
        high = 'high'
        low = 'low'
        volume = 'volume'
        
        tech_indicators = TechnicalIndicators()
        df = self.df.copy()
        
        df['MA'] = tech_indicators.simple_moving_avg(df, close, 15)
        df['+MA'] = df['MA'][df['MA'].diff() > 0]
        df['-MA'] = df['MA'][df['MA'].diff() < 0]
        df['EMA 10'] = tech_indicators.exponential_moving_avg(df, close, 10)
        df['EMA 20'] = tech_indicators.exponential_moving_avg(df, close, 20)
        df['+EMA 10'] = df['EMA 10'][df['EMA 10'] > df['EMA 20']]
        df['-EMA 10'] = df['EMA 10'][df['EMA 10'] < df['EMA 20']]
        df['MACD Line'] = tech_indicators.macd_line(df, close)
        df['MACD Signal'] = tech_indicators.macd_signal(df, close)
        df['+MACD Line'] = df['MACD Line'][df['MACD Line'] > df['MACD Signal']]
        df['-MACD Line'] = df['MACD Line'][df['MACD Line'] < df['MACD Signal']]
        df['RSI'] = tech_indicators.rsi(df, close)
        df['+RSI'] = df['RSI'][df['RSI'] < 30]
        df['-RSI'] = df['RSI'][df['RSI'] > 70]
        df['Stochastic Oscillator %K'] = tech_indicators.stochastic_oscillator_k(df, close)
        df['Stochastic Oscillator %D'] = tech_indicators.stochastic_oscillator_d(df, 'Stochastic Oscillator %K')
        df['+Stochastic Oscillator'] = df['Stochastic Oscillator %K'][np.logical_or(df['Stochastic Oscillator %K'] < 20, np.logical_and(np.logical_not(df['Stochastic Oscillator %K'] > 80), df['Stochastic Oscillator %K'] > df['Stochastic Oscillator %D']))]
        df['-Stochastic Oscillator'] = df['Stochastic Oscillator %K'][np.logical_or(df['Stochastic Oscillator %K'] > 80, np.logical_and(np.logical_not(df['Stochastic Oscillator %K'] < 20), df['Stochastic Oscillator %K'] < df['Stochastic Oscillator %D']))]
        df['CCI'] = tech_indicators.cci(df, close, high, low)
        df['+CCI'] = df['CCI'][df['CCI'] < -100]
        df['-CCI'] = df['CCI'][df['CCI'] > 100]
        df['Money Flow'] = tech_indicators.money_flow(df, close, high, low, volume)
        return df

In [1]:
class MPlot:
    '''Plot using matplotlib and pandas only'''
    def plot(self, stock):
        self._plot_data(stock.indicator_df, stock.symbol)
    
    def _plot_data(self, df, symbol):
        plt.style.use('seaborn')
        fig, axes = plt.subplots(6, 1)

        df.plot(ax=axes[0], y=['MA', '+MA', '-MA'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(11,18), title=symbol)
        df.plot(ax=axes[1], y=['EMA 10', '+EMA 10', '-EMA 10'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(11,18), title='Exponential Moving Average')
        df.plot(ax=axes[2], y=['MACD Signal', '+MACD Line', '-MACD Line'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(11,18), title='MACD')
        df.plot(ax=axes[3], y=['RSI', '+RSI', '-RSI'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(11,18), title='RSI')
        df.plot(ax=axes[4], y=['Stochastic Oscillator %K', '+Stochastic Oscillator', '-Stochastic Oscillator'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(10,18), title='Stochastic Oscillator')
        df.plot(ax=axes[5], y=['CCI', '+CCI', '-CCI'], color=['b', 'g', 'r'], linewidth=0.6, figsize=(11,18), title='CCI')

In [14]:
class BokehPlot:
    '''Plot using bokeh'''
    def __init__(self):
        self.indicators = ['Money Flow', 'MACD Line', 'RSI', 'Stochastic Oscillator %K', 'CCI']
    
    def plot(self, stock):
        self._plot_bokeh_data(stock.indicator_df);
        
    def _plot_bokeh_data(self, df):
        
        x_range = (df.index[-1000], df.index[-1])
        
        select = figure(title="Drag the middle and edges of the selection box to change the range above",
                plot_height=130, plot_width=800, x_range=x_range,
                x_axis_type="datetime", y_axis_type=None,
                tools="", toolbar_location=None, background_fill_color="#efefef")

#         range_tool = RangeTool(x_range=p.x_range)
#         range_tool.overlay.fill_color = "navy"
#         range_tool.overlay.fill_alpha = 0.2
        source = ColumnDataSource(data=dict(x=df.index, y=df['close']))
        select.line('x', 'y', source=source)
        select.ygrid.grid_line_color = None
#         select.add_tools(range_tool)
#         select.toolbar.active_multi = range_tool
        
        plots = []
        indicator_buy_periods = dict()
        
        for indicator in self.indicators:
            source = ColumnDataSource(data=dict(x=df.index, y=df[indicator]))

            p = figure(plot_height=300, plot_width=800,
               x_axis_type="datetime", x_axis_location="above", tools='pan,wheel_zoom,reset',
               background_fill_color="#efefef", x_range=select.x_range)

            p.line('x', 'y', source=source)
            p.yaxis.axis_label = 'Price'
            p.xaxis.axis_label = indicator
            p.add_tools(HoverTool(
                tooltips=[
                    ( 'date', '@x{%F}'),
                    ( 'y',  '@y'), # use @{ } for field names with spaces
                ],
                formatters={
                    'x': 'datetime', # use 'datetime' formatter for 'date' field
                },
                # display a tooltip whenever the cursor is vertically in line with a glyph
                mode='vline'
            ))
                
            periods = self._get_buy_sell_period_columns(df, indicator)

            for buy in periods['buy_periods']:
                p.add_layout(BoxAnnotation(left=buy[0], right =buy[1], fill_alpha=0.1, fill_color='green', line_color='green'))

            for sell in periods['sell_periods']:
                p.add_layout(BoxAnnotation(left=sell[0], right =sell[1], fill_alpha=0.1, fill_color='red', line_color='red'))
            # p.add_layout(BoxAnnotation(top=80, fill_alpha=0.1, fill_color='red', line_color='red'))

            plots.append(p)
            
            indicator_buy_periods[indicator] = periods['buy_periods']

        self._print_holding_stats(indicator_buy_periods, df)
#         show(column(p, select))
        show(column(plots))

    
    def _get_buy_sell_period_columns(self, df, indicator_name):
        buy_periods = []
        sell_periods = []
        i = 1
        df_length = len(df)
        while(i < df_length):
            if df[indicator_name][i] > df[indicator_name][i-1]:
                start = df.index[i]
                while(i < df_length and self._bokeh_period_conditions(df, indicator_name, i, 'buy')):
                    i+=1
                end = df.index[i-1]
                if (start < end):
                    buy_periods.append([start, end])
            elif df[indicator_name][i] < df[indicator_name][i-1]:
                start = df.index[i]
                while(i < df_length and self._bokeh_period_conditions(df, indicator_name, i, 'sell')):
                    i+=1
                end = df.index[i-1]
                if (start < end):
                    sell_periods.append([start, end])
            i+=1
        return {'buy_periods': buy_periods, 'sell_periods': sell_periods}
    
    def _bokeh_period_conditions(self, df, indicator_name, i, buy_sell):
        if buy_sell == 'buy':
            if indicator_name == 'MA':
                return df[indicator_name][i] > df[indicator_name][i - 1]
            elif indicator_name == 'EMA 10':
                return df['EMA 10'][i] > df['EMA 20'][i]
            elif indicator_name == 'MACD Line':
                return df['MACD Line'][i] > df['MACD Signal'][i]
            elif indicator_name == 'RSI':
                return df['RSI'][i] < 30
            elif indicator_name == 'Stochastic Oscillator %K':
                return df['Stochastic Oscillator %K'][i] < 20 or ((not df['Stochastic Oscillator %K'][i] > 80) and df['Stochastic Oscillator %K'][i] > df['Stochastic Oscillator %D'][i])
            elif indicator_name == 'CCI':
                return df['CCI'][i] < -100
            elif indicator_name == 'Money Flow':
                return False
        elif buy_sell == 'sell':
            if indicator_name == 'MA':
                return df[indicator_name][i] < df[indicator_name][i - 1]
            elif indicator_name == 'EMA 10':
                return df['EMA 10'][i] < df['EMA 20'][i]
            elif indicator_name == 'MACD Line':
                return df['MACD Line'][i] < df['MACD Signal'][i]
            elif indicator_name == 'RSI':
                return df['RSI'][i] > 70
            elif indicator_name == 'Stochastic Oscillator %K':
                return df['Stochastic Oscillator %K'][i] >80 or ((not df['Stochastic Oscillator %K'][i] < 20) and df['Stochastic Oscillator %K'][i] < df['Stochastic Oscillator %D'][i])
            elif indicator_name == 'CCI':
                return df['CCI'][i] > 100
            elif indicator_name == 'Money Flow':
                return False
            
    def _print_holding_stats(self, periods, df):
        for indicator in periods:
            print("\n For " + indicator + ": ")
            num_hold_periods = len(periods[indicator])
            print("Number of holding periods = " + str(num_hold_periods))
            total_hold_period = 0
            total_return = 0
            for period in periods[indicator]:
                total_hold_period += ((period[1] - period[0]).days)
                total_return += (df.loc[period[1]]['close'] - df.loc[period[0]]['close'])
            
            if num_hold_periods != 0:
                avg_hold_period = total_hold_period / num_hold_periods
                print("Average holding period = " + str(avg_hold_period) + " days")
                avg_return_per_period = total_return / num_hold_periods
                print("Average return per period = "+ str(avg_return_per_period))

In [12]:
apple = Stock('AAPL')

In [ ]:
google = Stock('GOOGL')

In [ ]:
facebook = Stock('FB')

In [ ]:
hp = Stock('HPQ')

In [ ]:
citi = Stock('C')

In [ ]:
aig = Stock('AIG')

In [20]:
apple.indicator_df

open      high       low   close    volume          MA  \
1998-01-02   13.63   16.2500   13.5000   16.25   6411700         NaN   
1998-01-05   16.50   16.5600   15.1900   15.88   5820300         NaN   
1998-01-06   15.94   20.0000   14.7500   18.94  16182800         NaN   
1998-01-07   18.81   19.0000   17.3100   17.50   9300200         NaN   
1998-01-08   17.44   18.6200   16.9400   18.19   6910900         NaN   
1998-01-09   18.12   19.3700   17.5000   18.19   7915600         NaN   
1998-01-12   17.44   18.6200   17.1200   18.25   4610700         NaN   
1998-01-13   18.62   19.6200   18.5000   19.50   5686200   18.478000   
1998-01-14   19.87   19.9400   19.2500   19.75   5261300   18.690667   
1998-01-15   19.19   19.7500   18.6200   19.19   4993500   18.906667   
1998-01-16   19.44   19.4400   18.6900   18.81   2199600   18.923333   
1998-01-20   19.06   19.3100   18.6200   19.06   2156800   18.990000   
1998-01-21   18.75   19.0600   18.5600   18.91   1698300   18.998000   
1998-01-22   18.69   19.7500   18.6200   19.25   2944000   18.964667   
1998-01-23   19.37   19.6900   19.2500   19.50   2081800   18.968667   
1998-01-26   19.44   19.5600   18.8100   19.44   1307500   18.885333   
1998-01-27   19.19   19.6900   19.0000   19.12   1002100   18.789333   
1998-01-28   19.19   19.3700   18.6200   19.19   1349300   18.743333   
1998-01-29   18.94   19.1200   18.5000   18.50   1891800   18.768667   
1998-01-30   18.31   18.8700   18.2500   18.31   1450400   18.794000   
1998-02-02   18.50   18.5000   17.3700   17.69   5685200   18.800000   
1998-02-03   17.69   18.6200   17.6900   18.31   3594800   18.808000   
1998-02-04   18.06   18.5000   18.0000   18.25   1519600   18.808000   
1998-02-05   18.25   18.5000   18.0000   18.31   2127400   18.820000   
1998-02-06   18.37   18.6900   18.2500   18.50   1806600   18.916000   
1998-02-09   18.37   19.5000   18.3700   19.19   4416700   18.999333   
1998-02-10   19.12   19.5600   19.0600   19.44   3768000   19.099333   
1998-02-11   19.50   19.5000   18.8700   19.00   1889900   19.295333   
1998-02-12   19.12   19.4400   19.0600   19.37   1819200   19.536667   
1998-02-13   19.19   19.8700   19.0000   19.50   1857100   19.803333   
...            ...       ...       ...     ...       ...         ...   
2019-02-04  167.41  171.6550  167.2800  171.25  31495582  166.078000   
2019-02-05  172.86  175.0800  172.3501  174.18  36101628  167.284667   
2019-02-06  174.65  175.5700  172.8531  174.24  28239591  168.128667   
2019-02-07  172.40  173.9400  170.3400  170.94  31741690  169.104000   
2019-02-08  168.99  170.6600  168.4200  170.41  23819966  170.260667   
2019-02-11  171.05  171.2100  169.2500  169.43  20993425  170.648000   
2019-02-12  170.10  171.0000  169.7000  170.89  22283523  171.083333   
2019-02-13  171.39  172.4800  169.9200  170.18  22490233  171.597333   
2019-02-14  169.71  171.2615  169.3800  170.80  21835747  171.802667   
2019-02-15  171.25  171.7000  169.7500  170.42  24626814  171.848667   
2019-02-19  169.71  171.4400  169.4900  170.93  18972826  171.776000   
2019-02-20  171.19  173.3200  170.9900  172.03  26114362  172.044667   
2019-02-21  171.80  172.3700  170.3000  171.06  17249670  172.407333   
2019-02-22  171.58  173.0000  171.3800  172.97  18913154  172.814000   
2019-02-25  174.16  175.8700  173.9500  174.23  21873358  173.056000   
2019-02-26  173.71  175.3000  173.1732  174.33  17070211  173.210667   
2019-02-27  173.21  175.0000  172.7300  174.87  27835389  173.351333   
2019-02-28  174.32  174.9100  172.9200  173.15  28215416  173.916667   
2019-03-01  174.28  175.1500  172.8900  174.97  25886167  174.582000   
2019-03-04  175.69  177.7500  173.9700  175.85  27436203  175.227333   
2019-03-05  175.94  176.0000  174.5400  175.53  19737419  176.072000   
2019-03-06  174.67  175.4900  173.9400  174.52  20810384  176.948667   
2019-03-07  173.87  174.4400  172.0200  172.50  24796374  177.868000   
2019-03-08  170.32  173.0700  169.5000  172.91  239993

In [15]:
bokeh_plot = BokehPlot()
bokeh_plot.plot(apple)


 For Money Flow: 
Number of holding periods = 0

 For MACD Line: 
Number of holding periods = 176
Average holding period = 20.40340909090909 days
Average return per period = 4.457576136363636

 For RSI: 
Number of holding periods = 26
Average holding period = 6.5 days
Average return per period = -4.757380769230769

 For Stochastic Oscillator %K: 
Number of holding periods = 344
Average holding period = 3.7005813953488373 days
Average return per period = 1.3512360465116269

 For CCI: 
Number of holding periods = 8
Average holding period = 1.75 days
Average return per period = 0.9874999999999972


In [ ]:
bokeh_plot.plot(google)

In [ ]:
bokeh_plot.plot(facebook)

In [ ]:
bokeh_plot.plot(hp)

In [ ]:
bokeh_plot.plot(citi)

In [ ]:
bokeh_plot.plot(aig)